In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'd:\\MLops\\End-to-End-DataScienceProject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name:str
    alpha: float
    l1_ratio: float
    target_column: str
    

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath= SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    def get_model_tarin_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params= self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([self.config.artifacts_root])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )
        return model_trainer_config


In [12]:
import pandas as pd
from src.datascience import logger
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib


In [13]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config
        self.train_data = pd.read_csv(config.train_data_path)
        self.test_data = pd.read_csv(config.test_data_path)
        self.model = ElasticNet(alpha=config.alpha,l1_ratio=config.l1_ratio)
        self.scaler = StandardScaler()
        
    def _preprocess_data(self):
        X = self.train_data.drop(columns=[self.config.target_column])
        y = self.train_data[self.config.target_column]
        
        X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=42)
        
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_val_scaled = self.scaler.transform(X_val)
        
        return X_train_scaled, X_val_scaled, y_train, y_val
    
    def train(self):
        X_train_scaled, X_val_scaled, y_train, y_val = self._preprocess_data()
        self.model.fit(X_train_scaled,y_train)
        
        train_preds = self.model.predict(X_train_scaled)
        val_preds = self.model.predict(X_val_scaled)
        
        train_rmse = mean_squared_error(y_train,train_preds,squared=False)
        val_rmse = mean_squared_error(y_val,val_preds,squared=False)
        
        logger.info(f"Train RMSE: {train_rmse}")
        logger.info(f"Validation RMSE: {val_rmse}")
        
        joblib.dump(self.model,self.config.root_dir / f"{self.config.model_name}.joblib")
        joblib.dump(self.scaler,self.config.root_dir / f"{self.config.model_name}_scaler.joblib")
        
        logger.info(f"Model and Scaler saved at {self.config.root_dir}")
        
        return train_rmse, val_rmse

In [14]:
try:
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_tarin_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.error(e)
    raise


[2025-01-25 14:25:29,087: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2025-01-25 14:25:29,090: INFO: common: yaml file: params.yaml loaded sucessfully]
[2025-01-25 14:25:29,092: INFO: common: yaml file: schema.yaml loaded sucessfully]
[2025-01-25 14:25:29,094: INFO: common: Craeted directory at:artifacts]
[2025-01-25 14:25:29,096: INFO: common: Craeted directory at:artifacts]
[2025-01-25 14:25:29,126: ERROR: 1666941101: got an unexpected keyword argument 'squared']


TypeError: got an unexpected keyword argument 'squared'